In [85]:
import random
from itertools import permutations
import math

In [86]:
class Problem:
    """Esta es una clase abstracta para un problema.
    No se supone que debamos crear objetos de esta
    clase directamente. La idea es que otras clases
    hereden de esta para crear clases de problemas
    más especeificos."""

    def random_solution(self):
        raise NotImplementedError

    def neighbors_of(self, state):
        raise NotImplementedError
    
    def goal_test(self, state):
        """De seguro ya te diste cuenta que todos estos
        métodos son muy generales y poco específicos.
        ¿Qué forma tienen los estados?
        ¿Qué son las acciones?
        ¿Qué es una meta?
        Este método regresa True si el estado dado es una
        meta. Por defecto se compara el estado dado con
        self.goal o verifica que el estado dado sea
        elemento de self.goal en caso de ser lista."""
        if isinstance(self.goal, list):
            return any(x is state for x in self.goal)
        return state == self.goal
    
    
    def all_domain(self):
        return NotImplementedError
    
    def random_neighbor(self, state):
        raise NotImplementedError
    
    def crossover(self,state1,state2):
        raise NotImplementedError
    
    def value(self, state):
        """En distintos problemas, cada estado tiene un valor asociado.
        Este método sirve para obtener este valor."""
        raise NotImplementedError

In [87]:
def factorial(num):
        if num == 1:
            return num
        else:
            return factorial(num-1) * num

In [88]:
class NReinas(Problem):
    def __init__(self,n_col):
        self.n_col = n_col
        domain = factorial(self.n_col)
        self.domain = domain

    def random_solution(self):
        solution = list(range(self.n_col))
        random.shuffle(solution)
        return solution

    def value(self, state):
        diagonal1 = []
        diagonal2 = []
        for i in range(self.n_col):
            diagonal1.append(i+state[i])
            diagonal2.append(i-state[i])
        return 2*self.n_col - (len(set(diagonal1)) + len(set(diagonal2)))
    
    def neighbors_of(self,state):
        neighbors = []
        for i in range(len(state)):
            for j in range(i+1,len(state)):
                neighbors.append(state[:i] + state[j:j+1] + state [i+1:j] + state[i:i+1] + state[j+1:])
        return neighbors

    def domain_size(self):
        if self.domain > 10000000:
            return True
        else:
            return False
    
    def all_domain(self):
        all = [i for i in range(self.n_col)]
        return list(permutations(all))
    
    def crossover(self, s1, s2):
        i = random.randint(1, len(s1)-2)
        state2 = [elem for elem in s2 if elem not in s1[:i]]
        return s1[:i] + state2

    def random_neighbor(self, state):
        lista = [i for i in range(self.n_col)]
        x = lista.pop(random.randint(0,len(lista)-1))
        y = lista.pop(random.randint(0,len(lista)-1))
        if x < y:
            return state[0:x] + [state[y]] + state[x+1:y] + [state[x]] + state[y+1:]
        else:
            return state[0:y] + [state[x]] + state[y+1:x] + [state[y]] + state[x+1:]
        


In [89]:
class OptimazationMethod:
    def solve(slef, Problem):
        raise NotImplementedError

In [90]:
class ExhaustiveSearch(OptimazationMethod):
    def solve(self,Problem):
        if Problem.domain_size():
            return "Impossible to calculate with this optimization method in this computer."
        else:
            solution = []
            for element in Problem.all_domain():
                if Problem.value(element) == 0:
                    solution.append(element)
        return solution

In [91]:
class SolveRandomly(OptimazationMethod):
    def __init__(self, repeats: int = 1000):
        self.repeats = repeats
        
    def solve(self,Problem,):
        best_cost = float('inf')
        best_sol = None
    
        for _ in range(self.repeats):
            s = Problem.random_solution()
            c = Problem.value(s)
            if c <= best_cost:
                best_cost = c
                best_sol = s
    
        return best_sol, best_cost

In [92]:
class HillClimbing(OptimazationMethod):
    def solve(self,Problem):
        s = Problem.random_solution()

        while True:
            cost = Problem.value(s)
            neighbors = Problem.neighbors_of(s)
            best_neighbor = min(neighbors, key=Problem.value)
            neighbor_cost = Problem.value(best_neighbor)
            if cost <= neighbor_cost:
                return s, cost
            
            s = best_neighbor     

In [93]:
class Annealing(OptimazationMethod):
    def __init__(self,Ti=10000,Tf=.1,alpha=.95):
        self.Ti = Ti
        self.Tf = Tf
        self.alpha = alpha

    def solve(self,Problem):
        solution = Problem.random_solution()
        cost = Problem.value(solution)
        T=self.Ti
        while T > self.Tf:
            if Problem.neighbors_of(solution) == 0:
                return solution
            neighbor = Problem.random_neighbor(solution)
            neighbor_cost = Problem.value(neighbor)
            diff = cost - neighbor_cost
            if diff > 0 or random.random() < math.exp(diff / T):
                solution = neighbor
                cost = neighbor_cost
            T = self.alpha*T
        return solution, cost

In [94]:
class Evoling(OptimazationMethod):
    def __init__(self, pop_size=50, mut_prob=0.2, elite=0.2, epochs=100):
        self.pop_size = pop_size
        self.mut_prob = mut_prob
        self.elite = elite
        self.epochs = epochs

    def solve(self, Problem):
        pop = [Problem.random_solution() for _ in range(self.pop_size)]
        top_elite = int(self.elite * self.pop_size)
        
        for epoch in range(self.epochs):
            pop.sort(key=Problem.value)
            best = pop[:top_elite]
            while len(best) < self.pop_size:
                if random.random() < self.mut_prob:
                    best.append(Problem.random_neighbor(
                        best[random.randint(0, top_elite-1)]
                    ))
                else:
                    best.append(Problem.crossover(
                        best[random.randint(0, top_elite-1)],
                        best[random.randint(0, top_elite-1)],
                    ))
            pop = best
        pop.sort(key=Problem.value)
        return pop[0], Problem.value(pop[0])

In [95]:
ReinasN = NReinas(8)

In [96]:
print(ExhaustiveSearch().solve(ReinasN))

[(0, 4, 7, 5, 2, 6, 1, 3), (0, 5, 7, 2, 6, 3, 1, 4), (0, 6, 3, 5, 7, 1, 4, 2), (0, 6, 4, 7, 1, 3, 5, 2), (1, 3, 5, 7, 2, 0, 6, 4), (1, 4, 6, 0, 2, 7, 5, 3), (1, 4, 6, 3, 0, 7, 5, 2), (1, 5, 0, 6, 3, 7, 2, 4), (1, 5, 7, 2, 0, 3, 6, 4), (1, 6, 2, 5, 7, 4, 0, 3), (1, 6, 4, 7, 0, 3, 5, 2), (1, 7, 5, 0, 2, 4, 6, 3), (2, 0, 6, 4, 7, 1, 3, 5), (2, 4, 1, 7, 0, 6, 3, 5), (2, 4, 1, 7, 5, 3, 6, 0), (2, 4, 6, 0, 3, 1, 7, 5), (2, 4, 7, 3, 0, 6, 1, 5), (2, 5, 1, 4, 7, 0, 6, 3), (2, 5, 1, 6, 0, 3, 7, 4), (2, 5, 1, 6, 4, 0, 7, 3), (2, 5, 3, 0, 7, 4, 6, 1), (2, 5, 3, 1, 7, 4, 6, 0), (2, 5, 7, 0, 3, 6, 4, 1), (2, 5, 7, 0, 4, 6, 1, 3), (2, 5, 7, 1, 3, 0, 6, 4), (2, 6, 1, 7, 4, 0, 3, 5), (2, 6, 1, 7, 5, 3, 0, 4), (2, 7, 3, 6, 0, 5, 1, 4), (3, 0, 4, 7, 1, 6, 2, 5), (3, 0, 4, 7, 5, 2, 6, 1), (3, 1, 4, 7, 5, 0, 2, 6), (3, 1, 6, 2, 5, 7, 0, 4), (3, 1, 6, 2, 5, 7, 4, 0), (3, 1, 6, 4, 0, 7, 5, 2), (3, 1, 7, 4, 6, 0, 2, 5), (3, 1, 7, 5, 0, 2, 4, 6), (3, 5, 0, 4, 1, 7, 2, 6), (3, 5, 7, 1, 6, 0, 2, 4), (3, 5, 7, 2

In [97]:
print(SolveRandomly().solve(ReinasN))

([6, 1, 3, 0, 7, 4, 2, 5], 0)


In [98]:
print(HillClimbing().solve(ReinasN))

([4, 6, 0, 3, 1, 7, 5, 2], 0)


In [99]:
print(Annealing().solve(ReinasN))

([5, 6, 1, 3, 0, 7, 4, 2], 1)


In [100]:
print(Evoling().solve(ReinasN))

([1, 4, 6, 3, 0, 7, 5, 2], 0)
